In [1]:
#default_exp format
%load_ext autoreload
%autoreload 2

In [3]:
import sys
from pathlib import Path
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

In [4]:
#export
import re
from typing import List, Iterable, Union
from pathlib import Path
from collections import OrderedDict
from decimal import Decimal

import pandas as pd
from unidecode import unidecode
from gazpacho import Soup
import requests
from fastcore.utils import listify
from rich.progress import track

from anateldb.constants import ENTIDADES, COL_PB, ESTACAO, BW, BW_pattern

# Formatação 

> Este módulo possui funções auxiliares de formatação dos dados das várias fontes.

In [ ]:
#export
def row2dict(row: dict) -> dict:  # sourcery skip: identity-comprehension
    """Receives a json row and return the dictionary from it"""
    return dict(row.items())


def dict2cols(df: pd.DataFrame, reject: Iterable[str] = ()) -> pd.DataFrame:
    """Recebe um dataframe com dicionários nas células e extrai os dicionários como colunas
    Opcionalmente ignora e exclue as colunas em reject
    """
    for column in df.columns:
        if column in reject:
            df.drop(column, axis=1, inplace=True)
            continue
        if type(df[column].iloc[0]) == OrderedDict:
            try:
                new_df = pd.DataFrame(df[column].apply(row2dict).tolist())
                df = pd.concat([df, new_df], axis=1)
                df.drop(column, axis=1, inplace=True)
            except AttributeError:
                continue
    return df


def parse_plano_basico(row: dict, cols: Iterable[str] = COL_PB) -> dict:
    """Receives a json row and filter the column in `cols`"""
    return {k: row[k] for k in cols}


def scrape_dataframe(id_list: Iterable[str]) -> pd.DataFrame:
    """Receives a list of ids and returns a dataframe with the data from web scraping the MOSAICO page"""
    df = pd.DataFrame()
    for id_ in track(id_list, description="Baixando informações complementares da Web"):
        html = requests.get(ESTACAO.format(id_))
        df = df.append(
            pd.read_html(Soup(getattr(html, "text", "")).find("table").html)[0]
        )

    df.rename(
        columns={"NumFistel": "Fistel", "Num Serviço": "Num_Serviço"}, inplace=True
    )
    return df[
        [
            "Status",
            "Entidade",
            "Fistel",
            "Frequência",
            "Classe",
            "Num_Serviço",
            "Município",
            "UF",
        ]
    ]

In [ ]:
#export
def input_coordenates(df: pd.DataFrame, pasta: Union[str, Path]) -> pd.DataFrame:
    """Input the NA's in Coordinates with those of the cities"""
    municipios = Path(f"{pasta}/municípios.fth")
    if not municipios.exists():
        municipios = Path(f"{pasta}/municípios.xlsx")
        if not municipios.exists():
            raise FileNotFoundError(
                f"É necessario a tabela de municípios municípios.fth | municípios.xlsx na pasta {pasta}"
            )
        m = pd.read_excel(municipios, engine="openpyxl")
    else:
        m = pd.read_feather(municipios)
    m.loc[
        m.Município == "Sant'Ana do Livramento", "Município"
    ] = "Santana do Livramento"
    m["Município"] = m.Município.apply(unidecode).str.lower().str.replace("'", " ")
    m["UF"] = m.UF.str.lower()
    df["Coordenadas_do_Município"] = False
    df["Latitude"] = df.Latitude.str.replace(",", ".")
    df["Longitude"] = df.Longitude.str.replace(",", ".")
    df.loc[df["Município"] == "Poxoréo", "Município"] = "Poxoréu"
    df.loc[df["Município"] == "Couto de Magalhães", "Município"] = "Couto Magalhães"
    df["Município"] = df.Município.astype("string")
    criteria = (
        (df.Latitude == "")
        | (df.Latitude.isna())
        | (df.Longitude == "")
        | (df.Longitude.isna())
    ) & df.Município.isna()
    df = df[~criteria]
    for row in df[
        (
            (df.Latitude == "")
            | (df.Latitude.isna())
            | (df.Longitude == "")
            | (df.Longitude.isna())
        )
    ].itertuples():
        try:
            left = unidecode(row.Município).lower()
            m_coord = (
                m.loc[
                    (m.Município == left) & (m.UF == row.UF.lower()),
                    ["Latitude", "Longitude"],
                ]
                .values.flatten()
                .tolist()
            )
            df.loc[row.Index, "Latitude"] = m_coord[0]
            df.loc[row.Index, "Longitude"] = m_coord[1]
            df.loc[row.Index, "Coordenadas_do_Município"] = True
        except ValueError:
            print(left, row.UF, m_coord)
            continue
    return df


def parse_bw(bw: str) -> float:
    """Parse the bandwidth string"""
    if match := re.match(BW_pattern, bw):
        multiplier = BW[match.group(2)]
        if mantissa := match.group(3):
            number = float(f"{match.group(1)}.{mantissa}")
        else:
            number = float(match.group(1))
        return multiplier * number
    return -1

## Otimização dos Tipos de dados
A serem criados dataframes, normalmente a tipo de data é aquele com maior resolução possível, nem sempre isso é necessário, os arquivos de espectro mesmo possuem somente uma casa decimal, portanto um `float16` já é suficiente para armazená-los. As funções a seguir fazem essa otimização

Code below borrowed from https://medium.com/bigdatarepublic/advanced-pandas-optimize-speed-and-memory-a654b53be6c2

In [ ]:
#export
def optimize_floats(df: pd.DataFrame, exclude: Iterable[str] = None) -> pd.DataFrame:
    """Optimize the floats in the dataframe to reduce the memory usage"""
    floats = df.select_dtypes(include=["float64"]).columns.tolist()
    floats = [c for c in floats if c not in listify(exclude)]
    df[floats] = df[floats].apply(pd.to_numeric, downcast="float")
    return df


def optimize_ints(df: pd.DataFrame, exclude: Iterable[str] = None) -> pd.DataFrame:
    """Optimize the ints in the dataframe to reduce the memory usage"""
    ints = df.select_dtypes(include=["int64"]).columns.tolist()
    ints = [c for c in ints if c not in listify(exclude)]
    df[ints] = df[ints].apply(pd.to_numeric, downcast="integer")
    return df


def optimize_objects(
    df: pd.DataFrame,
    datetime_features: Iterable[str] = None,
    exclude: Iterable[str] = None,
) -> pd.DataFrame:
    """Optimize the objects in the dataframe to category | string to reduce the memory usage"""
    exclude = listify(exclude)
    datetime_features = listify(datetime_features)
    for col in df.select_dtypes(
        include=["object", "string", "category"]
    ).columns.tolist():
        if col not in datetime_features:
            if col in exclude:
                continue
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if float(num_unique_values) / num_total_values < 0.5:
                dtype = "category"
            else:
                dtype = "string"
            df[col] = df[col].astype(dtype)
        else:
            df[col] = pd.to_datetime(df[col]).dt.date
    return df


def df_optimize(
    df: pd.DataFrame,
    datetime_features: Iterable[str] = None,
    exclude: Iterable[str] = None,
):
    """Optimize the data types in dataframe to reduce the memory usage"""
    if datetime_features is None:
        datetime_features = []
    return optimize_floats(
        optimize_ints(optimize_objects(df, datetime_features, exclude), exclude),
        exclude,
    )

In [5]:
from nbdev.export import notebook2script; notebook2script()

Converted Apresentação REFIS 2022 - anateldb.ipynb.
Converted constants.ipynb.
Converted filter.ipynb.
Converted format.ipynb.
Converted index.ipynb.
Converted merge.ipynb.
Converted query.ipynb.
Converted read.ipynb.
